# Revenue Prediction EDA

In [1]:
'''This notebook loads, explores, and visualizes the revenue prediction dataset'''

__author__ = 'Jaime Garvey'
__email__ = 'jaimemgarvey@gmail.com'

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px 
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly

%matplotlib inline

## Load Data

In [444]:
search_df_raw = pd.read_excel('../data/raw/sample data.xlsx') #save raw data
search_df = search_df_raw
search_df.set_index('date', inplace=True) #set date as index
search_df.head(2)

,revenue,google_search_Bottom,google_search_Mid,google_search_Top,bing_search_Bottom,bing_search_Top,facebook_Bottom,facebook_Mid,facebook_Top
date,,,,,,,,,
2019-01-01,30638.84,248.73,0.27,1286.40,17.23,49.940000,121.58,220.94,5416.81
2019-01-02,30134.13,209.05,1.94,1243.62,18.18,59.969999,120.67,216.65,5711.90


## Examine

In [323]:
search_df.info();

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 943 entries, 2019-01-01 to 2021-07-31
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   revenue               943 non-null    float64
 1   google_search_Bottom  943 non-null    float64
 2   google_search_Mid     943 non-null    float64
 3   google_search_Top     943 non-null    float64
 4   bing_search_Bottom    943 non-null    float64
 5   bing_search_Top       943 non-null    float64
 6   facebook_Bottom       943 non-null    float64
 7   facebook_Mid          943 non-null    float64
 8   facebook_Top          943 non-null    float64
dtypes: float64(9)
memory usage: 73.7 KB


No null variables and all numerical features. 

### Check for duplicate rows

In [324]:
print(f'There are {search_df.duplicated().sum()} duplicate rows.')

There are 0 duplicate rows.


### Summarize features

In [325]:
search_df.describe()

,revenue,google_search_Bottom,google_search_Mid,google_search_Top,bing_search_Bottom,bing_search_Top,facebook_Bottom,facebook_Mid,facebook_Top
count,9.430000e+02,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000
mean,2.722663e+05,2479.536227,7131.386664,2792.944688,127.914909,85.734708,1096.573955,4467.297275,35510.001792
std,2.360243e+05,2434.166013,8300.609037,1508.197840,121.996774,46.478454,756.162182,5296.174167,27264.571596
min,8.572000e+01,133.360000,0.000000,218.933189,8.159999,9.560000,36.280000,9.980000,4175.650000
25%,7.181149e+04,614.425000,46.705000,1625.509892,34.144999,53.765000,287.845000,416.460000,13930.070000
50%,2.321819e+05,1999.383493,4203.237547,2555.580000,79.360000,73.090000,1114.880000,2619.850000,28755.310000
75%,4.276580e+05,3463.135067,12697.212932,3709.101690,228.759999,98.625000,1764.080000,9838.480000,56722.310000
max,1.976391e+06,20938.564186,36742.228341,10628.200987,710.279998,312.319999,2908.950000,18910.580000,114022.850000


Note: There's a zero in the Google Search Mid feature. We will revisit this when we look for outliers. 

## Visualize Target (Revenue)

### Revenue Distribution

In [326]:
fig_rev = make_subplots(rows=1, cols=2)

fig_rev.add_trace(
    go.Box(y=search_df['revenue']),
    row=1, col=1
)

fig_rev.add_trace(
    go.Histogram(x=search_df['revenue']),
#     ff.create_distplot([search_df['revenue']], ['revenue']),
    row=1, col=2
)

fig_rev.update_layout(height=600, width=1000, title_text="Revenue (Target Variable) Distribution")
fig_rev.show()

### Total Spend vs. Revenue

In [445]:
#look at total spend vs revenue

search_df['total_spend']=search_df[search_df.columns.difference(['date', 'revenue'])].sum(axis=1)

fig = px.scatter(search_df, x="total_spend", y="revenue", title='Revenue vs. Total Search Spend')
fig.show()

### Revenue Time Series

We can see some seasonality and increasing trend in the data. Highest annual revenues are in late November/early December. 

In [451]:
#resample weekly 
weekly_avg = search_df.resample('W-MON').mean()
weekly_sum = search_df.resample('W-MON').sum()
#plot time series daily & weekly
fig = make_subplots(rows=2, cols=1, subplot_titles=('Daily', 'Weekly'))

fig.add_trace(go.Scatter(x=search_df.index, y=search_df['revenue']),row=1, col=1)
fig.add_trace(go.Scatter(x=weekly_avg.index, y=weekly_avg.revenue), row=2, col=1)

fig.update_layout(height=600, width=800, title_text="Revenue Time Series")
fig.show()

## Visualize Features

In [465]:
#Helper Functions
def group_cols(df, cols_desired):
    '''
    Helper function to group cols by platform or funnel stage

    example:
    {'google': ['google_search_Bottom', 'google_search_Mid',
       'google_search_Top']
    }
    '''
    grouped_dict = {}
    
    for group in cols_desired:
        grouped_dict[group] = [col for col in df.columns if group in col.lower()]
        
    return grouped_dict

def agg_groups(df, groups_dict):
    '''Sum group across row'''
    for key in list(grouped_cols.keys()):
        cols = groups_dict[key]
        df[key] = df[cols].sum(axis=1)
        
    return df

In [466]:
#create dictionary of groups and cols names
groups_dict = group_cols(search_df, 
                         ['google', 'facebook', 'bing', 'bottom', 'mid', 'top'] )
search_df = agg_groups(search_df, groups_dict) # add grouped columns to df

weekly_sum_temp = agg_groups(weekly_sum, groups_dict)
weekly_sum_temp['total_spend'] = weekly_sum['total_spend']
weekly_sum = weekly_sum_temp

# save different time intervals
# search_df['year'] = search_df['date'].dt.year
# search_df['quarter'] = search_df['date'].dt.quarter
# search_df['month'] =search_df['date'].dt.month
# search_df['week'] = search_df['date'].dt.isocalendar().week
# search_df['day'] = search_df['date'].dt.weekday

# full_year_df = search_df[search_df.date < '2021-01-01']

### Spend by All Paid Media Features

In [417]:
#visualize distribution of spend across all features
spend_features = pd.DataFrame(search_df_raw[search_df_raw.columns.difference(['revenue', 'date'])].sum().sort_values(ascending=False), columns=['spend']).rename_axis('paid_media').reset_index()
spend_features['pct'] = (spend_features['spend']/spend_features['spend'].sum()*100).round(1)

fig = px.bar(spend_features, x="paid_media", y="spend",text=spend_features.pct.astype(str) + '%', title="Spend by All Paid Media Features")
fig.show()

### Spending by Paid Media Platform

In [399]:
spend_pm = pd.DataFrame(search_df[['google', 'bing', 'facebook']].sum(), columns=['spend']).rename_axis('paid_media').reset_index()
spend_pm['pct'] = (spend_pm['spend']/spend_pm['spend'].sum()*100).round(1)

fig = px.bar(spend_pm, x="paid_media", y="spend",text=spend_pm.pct.astype(str) + '%', title="Spend by Paid Media")
fig.show()

### Distribution of Platform Spend Weekly

In [435]:
weekly_pm = search_df[['google', 'bing', 'facebook']].resample('W-MON').sum().round(2)
fig = px.area(weekly_pm, x=weekly_pm.index, y=weekly_pm.columns, title="Proportion of spend", color_discrete_sequence =['#F4B400','#2cd4e0', '#4267B2'])
fig.show()

In [463]:
#temp = weekly_sum[['google', 'bing','facebook']].div(weekly_sum['total_spend'], axis=0)

### Spend by Funnel Stage

In [468]:
rev_funnel = pd.DataFrame(search_df[['bottom', 'mid', 'top']].sum(), columns=['spend']).rename_axis('funnel').reset_index()

fig = px.bar(rev_funnel, y="funnel", x="spend", orientation='h')
fig.show()

### Distribution of Funnel Stage Spend Weekly

In [492]:
weekly_funnel = search_df[['bottom', 'mid','top']].resample('W-MON').sum().round(2)
fig = px.area(weekly_funnel, x=weekly_funnel.index, y=weekly_funnel.columns, title="Spend by Funnel Stage Time Series", color_discrete_sequence =['#3b4295','#636ef9', '#a1a8fb'])
fig.show()

In [491]:
temp = weekly_funnel[['bottom', 'mid','top']].div(weekly_sum['total_spend'], axis=0)
fig = px.bar(temp, x=temp.index, y=['top', 'mid','bottom'], title="Distribution of Spend by Funnel Stage Time Series")
fig.show()

Proportion of spend on Top of Funnel decreases as starting at the beginning of 2020. We also see that revenue had a positive trend over a similar time frame.

### Other Variables by Date

The shape of the other variables (facebook_Top and Google Search Mid) looks consistent with the target variable (Revenue)

In [540]:
fig = px.line(weekly_avg, x=weekly_avg.index, y=weekly_avg.columns.difference(['revenue','date','total_spend']))
fig.show()

### Explore Time Intervals

#### Preprocess & Helper Functions

In [534]:
# search_df['date'] = pd.to_datetime(search_df['date']) #convert date to datetime

# save different time intervals
search_df['year'] = search_df.index.year
search_df['quarter'] = search_df.index.quarter
search_df['month'] =search_df.index.month
search_df['week'] = search_df.index.isocalendar().week
search_df['day'] = search_df.index.weekday

full_year_df = search_df[search_df.index < '2021-01-01']

In [535]:
def plot_variable_by_time(variable='revenue', df=full_year_df):
    
    day = pd.DataFrame(full_year_df.groupby('day')[variable].sum()).reset_index()
    week= pd.DataFrame(full_year_df.groupby('week')[variable].sum()).reset_index()
    month = pd.DataFrame(full_year_df.groupby('month')[variable].sum()).reset_index()
    quarter = pd.DataFrame(full_year_df.groupby('quarter')[variable].sum()).reset_index()


    fig = make_subplots(rows=2, cols=2,  subplot_titles=("Day", "Week", "Month", "Quarter"))

    fig.add_trace(
        go.Bar(x=day.day.to_list(), y=day[variable].to_list()),
        row=1, col=1
    )

    fig.add_trace(
        go.Bar(x=week.week.to_list(), y=week[variable].to_list()),
        row=1, col=2
    )

    fig.add_trace(
        go.Bar(x=month.month.to_list(), y=month[variable].to_list()),
        row=2, col=1
    )

    fig.add_trace(
        go.Bar(x=quarter.quarter.to_list(), y=quarter[variable].to_list()),
        row=2, col=2
    )

    fig.update_layout(height=600, width=800, title_text=f"{variable.capitalize()} by Day, Week, Month, Quarter")
    return fig.show()

#### Visuals

In [537]:
plot_variable_by_time()

## Outliers

### Identify Potential Outliers (Using IQR)

In [58]:
stat = search_df.revenue.describe()
# print(stat)

IQR = stat['75%'] - stat['25%']
upper = stat['75%'] + 1.5 * IQR
lower = stat['25%'] - 1.5 * IQR

print(f'Outlier Upper Bound: {upper:.2f} \n Outlier Lower Bound: {lower:.2f}')

Outlier Upper Bound: 961427.68 
 Outlier Lower Bound: -461958.22


The IQR method identifies a few outliers but they do not look like data errors. For example, in the case of the revenue shock, the spend also showed a shock. 

We'll leave those observations in the dataset for now but we can explore removing them if needed using various methods (e.g., Kalman filters). 

## Pickle Dataframe

In [546]:
selected_cols = ['revenue', 'google_search_Bottom', 'google_search_Mid',
       'google_search_Top', 'bing_search_Bottom', 'bing_search_Top',
       'facebook_Bottom', 'facebook_Mid', 'facebook_Top', 'google', 'bing',
       'facebook', 'top', 'mid', 'bottom', 'total_spend']

search_df_select = search_df[selected_cols]

search_df_select.to_pickle('../data/processed/search_df_select.pkl')

search_df_select.to_csv('../data/processed/search_df_select.csv', index=True)